In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# ---- Load all CSV files ----
team_df = pd.read_csv('team.csv')
budgets_df = pd.read_csv('budgets.csv')
holidays_df = pd.read_csv('holidays.csv')
leaves_df = pd.read_csv('leaves.csv')
logged_df = pd.read_csv('logged.csv')

In [3]:
# ---- Clean column names ----
budgets_df.columns = budgets_df.columns.str.strip()
holidays_df.columns = holidays_df.columns.str.strip()
leaves_df.columns = leaves_df.columns.str.strip()
logged_df.columns = logged_df.columns.str.strip()

In [4]:
# ---- Switch: Set reporting window ----
from_date = 20250701  # YYYYMMDD
to_date = 20250715    # YYYYMMDD

In [5]:
# ---- Calculate total_days ----
from_dt = datetime.strptime(str(from_date), "%Y%m%d")
to_dt = datetime.strptime(str(to_date), "%Y%m%d")
total_days = (to_dt - from_dt).days + 1

In [6]:
# ---- Holidays in range ----
holidays_in_range = holidays_df[
    (holidays_df['day'] >= from_date) & (holidays_df['day'] <= to_date)
]
num_holidays = holidays_in_range.shape[0]

In [7]:
# ---- Leaves in range ----
leaves_in_range = leaves_df[
    (leaves_df['day'] >= from_date) & (leaves_df['day'] <= to_date)
]
leaves_count = leaves_in_range.groupby('id').size().reset_index(name='leaves')

In [8]:
# ---- Build base report ----
report_df = team_df.merge(leaves_count, on='id', how='left')
report_df['leaves'] = report_df['leaves'].fillna(0).astype(int)
report_df['holidays'] = num_holidays
report_df['total_days'] = total_days
report_df['work_days'] = report_df['total_days'] - report_df['holidays'] - report_df['leaves']

In [9]:
# ---- Logged: Total logged_days per ID ----
logged_days = logged_df.groupby('id')['logged'].sum().reset_index(name='logged_days')

report_df = report_df.merge(logged_days, on='id', how='left')
report_df['logged_days'] = report_df['logged_days'].fillna(0)

In [10]:
# ---- Pivot tasks: task1, task2, task3, task4 ----
task_pivot = logged_df.pivot_table(index='id', columns='task', values='logged', aggfunc='sum').reset_index()
task_pivot = task_pivot.fillna(0)

report_df = report_df.merge(task_pivot, on='id', how='left')

In [11]:
# ---- Ensure all task columns exist ----
for task in ['task1', 'task2', 'task3', 'task4']:
    if task not in report_df.columns:
        report_df[task] = 0

In [12]:
# ---- Balance days ----
report_df['balance_days'] = report_df['work_days'] - report_df['logged_days']

In [14]:
# ---- Final columns ----
final_cols = [
    'id', 'name', 'leaves', 'holidays', 'total_days', 'work_days',
    'logged_days', 'task1', 'task2', 'task3', 'task4', 'balance_days'
]
report_df = report_df[final_cols]

In [17]:
# ---- Display final report ----
print("\n=== Team Allocation Report ===\n")
report_df


=== Team Allocation Report ===



,id,name,leaves,holidays,total_days,work_days,logged_days,task1,task2,task3,task4,balance_days
0,p001,aaa,1,1,15,13,3.00,0,3.0,0.00,0.00,10.00
1,p002,bbb,1,1,15,13,7.93,0,0.0,3.56,4.37,5.07
2,p003,ccc,0,1,15,14,0.00,0,NaN,NaN,NaN,14.00
3,p004,ddd,0,1,15,14,0.00,0,NaN,NaN,NaN,14.00
4,p005,eee,0,1,15,14,0.00,0,NaN,NaN,NaN,14.00
5,p006,fff,0,1,15,14,0.00,0,NaN,NaN,NaN,14.00
6,p007,ggg,0,1,15,14,0.00,0,NaN,NaN,NaN,14.00
7,p008,hhh,0,1,15,14,0.00,0,NaN,NaN,NaN,14.00
8,p009,iii,0,1,15,14,0.00,0,NaN,NaN,NaN,14.00
9,p010,jjj,0,1,15,14,0.00,0,NaN,NaN,NaN,14.00


In [16]:
# ---- Show total budget for reference ----
total_budget = budgets_df['budget'].sum()
print(f"\nTotal Budget (in FTE terms): {total_budget}")


Total Budget (in FTE terms): 19.0
